# Covid-19 Data Wrangling in Python

## Fatality statistics and ranks among countries

In [1]:
import pandas as pd
import numpy as np

print('pandas version: ', pd.__version__)
print('numpy version: ', np.version.version)

pandas version:  1.0.1
numpy version:  1.18.1


In [2]:
covid_daily_df = pd.read_csv('data/2020-03-31.csv')

covid_daily_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-03-31 23:43:56,34.223334,-82.461707,4,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-03-31 23:43:56,30.295065,-92.414197,39,1,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-03-31 23:43:56,37.767072,-75.632346,7,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-03-31 23:43:56,43.452658,-116.241552,195,3,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-03-31 23:43:56,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [3]:
covid_daily_df.dtypes

FIPS              float64
Admin2             object
Province_State     object
Country_Region     object
Last_Update        object
Lat               float64
Long_             float64
Confirmed           int64
Deaths              int64
Recovered           int64
Active              int64
Combined_Key       object
dtype: object

In [4]:
covid_daily_df.shape

(2434, 12)

In [5]:
# convert the FIPS County Code column to type of integer

covid_daily_df.FIPS = covid_daily_df.FIPS.fillna(-1) # get rid of NA 
covid_daily_df.FIPS = covid_daily_df.FIPS.astype(int)

covid_daily_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001,Abbeville,South Carolina,US,2020-03-31 23:43:56,34.223334,-82.461707,4,0,0,0,"Abbeville, South Carolina, US"
1,22001,Acadia,Louisiana,US,2020-03-31 23:43:56,30.295065,-92.414197,39,1,0,0,"Acadia, Louisiana, US"
2,51001,Accomack,Virginia,US,2020-03-31 23:43:56,37.767072,-75.632346,7,0,0,0,"Accomack, Virginia, US"
3,16001,Ada,Idaho,US,2020-03-31 23:43:56,43.452658,-116.241552,195,3,0,0,"Ada, Idaho, US"
4,19001,Adair,Iowa,US,2020-03-31 23:43:56,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


In [6]:
# show all unique Country_Region codes

countries = covid_daily_df.Country_Region.unique()
print(countries)
len(countries)

['US' 'Canada' 'United Kingdom' 'China' 'Netherlands' 'Australia'
 'Denmark' 'France' 'Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Austria' 'Azerbaijan'
 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus' 'Belgium' 'Belize'
 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina' 'Botswana' 'Brazil'
 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi' 'Cabo Verde'
 'Cambodia' 'Cameroon' 'Central African Republic' 'Chad' 'Chile'
 'Colombia' 'Congo (Brazzaville)' 'Congo (Kinshasa)' 'Costa Rica'
 "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus' 'Czechia' 'Diamond Princess'
 'Djibouti' 'Dominica' 'Dominican Republic' 'Ecuador' 'Egypt'
 'El Salvador' 'Equatorial Guinea' 'Eritrea' 'Estonia' 'Eswatini'
 'Ethiopia' 'Fiji' 'Finland' 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana'
 'Greece' 'Grenada' 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti'
 'Holy See' 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran'
 'Iraq' 'Ireland' 'Israel' 'It

180

In [7]:
# statistics grouped by countries
covid_daily_df.groupby('Country_Region').agg({'Confirmed': np.sum})

,Confirmed
Country_Region,
Afghanistan,174
Albania,243
Algeria,716
Andorra,376
Angola,7
...,...
Venezuela,135
Vietnam,212
West Bank and Gaza,119


In [8]:
# construct a dataframe with five columns: 
# Country_Region (index), Confirmed, Deaths, Recovered, Active
df_country_cases = covid_daily_df.groupby('Country_Region').agg({'Confirmed': np.sum, 'Deaths': np.sum, 'Recovered': np.sum, 'Active': np.sum})

df_country_cases

,Confirmed,Deaths,Recovered,Active
Country_Region,,,,
Afghanistan,174,4,5,165
Albania,243,15,52,176
Algeria,716,44,46,626
Andorra,376,12,10,354
Angola,7,2,1,4
...,...,...,...,...
Venezuela,135,3,39,93
Vietnam,212,0,58,154
West Bank and Gaza,119,1,18,100


In [9]:
# show counties that do not have any corona patients yet
len(df_country_cases.loc[df_country_cases['Confirmed'] == 0])


0

In [10]:
# last operation tells that all countries in this csv file have confirmed cases. 
# now we can calculate the fatality rate

df_country_cases['fatality_rate'] = df_country_cases.Deaths / df_country_cases.Confirmed

fatalities = df_country_cases.query('(Deaths > 0) & (Confirmed > 30)') \
                            .sort_values(by = 'fatality_rate', ascending = False)

fatalities[:25]

,Confirmed,Deaths,Recovered,Active,fatality_rate
Country_Region,,,,,
Italy,105792,12428,15729,77635,0.117476
San Marino,236,26,13,197,0.110169
Bangladesh,51,5,25,21,0.098039
Indonesia,1528,136,81,1311,0.089005
Spain,95923,8464,19259,68200,0.088237
Netherlands,12667,1040,253,11374,0.082103
Congo (Kinshasa),98,8,2,88,0.081633
Iraq,694,50,170,474,0.072046
United Kingdom,25481,1793,179,23509,0.070366


In [11]:
fatalities_asc = df_country_cases.query('(Deaths > 0) & (Confirmed > 25)').sort_values(by = 'fatality_rate', ascending = True)

fatalities_asc[:25]

,Confirmed,Deaths,Recovered,Active,fatality_rate
Country_Region,,,,,
New Zealand,647,1,74,572,0.001546
Iceland,1135,2,198,935,0.001762
Qatar,781,2,62,717,0.002561
Uruguay,338,1,41,296,0.002959
Singapore,926,3,240,683,0.003240
South Africa,1353,5,31,1317,0.003695
Israel,5358,20,224,5114,0.003733
Australia,4559,18,358,4183,0.003948
Chile,2738,12,156,2570,0.004383


In [12]:
df_country_cases.Deaths.sum()

42107

In [13]:
# fatalitiy over all countries having more than 100 confirmed cases. 

fatality_partial = fatalities.Deaths.sum() / fatalities.Confirmed.sum()

fatality_partial

0.049239268799490817

In [14]:
# fatalitiy over all countries, regardless of how many confirmed cases and deaths. 

fatality_overall = df_country_cases.Deaths.sum() / df_country_cases.Confirmed.sum()

fatality_overall

0.04910511762860545

In [15]:
# define a function which returns the rank of the fatality

def fatality_rank( country: str, asc: bool ) -> int:
    indices = fatalities.sort_values(by = 'fatality_rate', ascending = asc).index
    return indices.get_loc(country)


In [16]:
ranks_asc = fatalities.sort_values(by = 'fatality_rate', ascending = True).index

def get_fatality_and_rank(country: str) -> pd.Series:
    fatality = fatalities.loc[country].fatality_rate

    rank_asc = ranks_asc.get_loc(country) + 1
    rank_dsc = len(ranks_asc) - rank_asc + 1

    return pd.Series({'Country_Region': country, 'fatality_rate': fatality, 'rank_asc': rank_asc, 'rank_dsc': rank_dsc})


In [17]:
# test the function get_fatality_and_rank

get_fatality_and_rank('Italy')

Country_Region       Italy
fatality_rate     0.117476
rank_asc               111
rank_dsc                 1
dtype: object

In [18]:
# how do these countries score in the fatality_rate?

cols = ['Country_Region', 'fatality_rate', 'rank_asc', 'rank_dsc']
countries = ['Austria', 'China', 'Germany', 'Italy', 'New Zealand', 'Singapore', 'US']

df_fatality_scores = pd.DataFrame(columns = cols)

for country in countries:
    df_fatality_scores = df_fatality_scores.append(get_fatality_and_rank(country), ignore_index=True)

df_fatality_scores

,Country_Region,fatality_rate,rank_asc,rank_dsc
0,Austria,0.012574,34,78
1,China,0.040217,88,24
2,Germany,0.010793,29,83
3,Italy,0.117476,111,1
4,New Zealand,0.001546,1,111
5,Singapore,0.003240,5,107
6,US,0.020582,54,58


In [19]:
# basic statistics of the fatality
fatalities.fatality_rate.describe()

count    111.000000
mean       0.028277
std        0.024592
min        0.001546
25%        0.010676
50%        0.021947
75%        0.034724
max        0.117476
Name: fatality_rate, dtype: float64

In [20]:
# median and mode of fatality rate

fatality_median = fatalities.fatality_rate.median()
fatality_mode = fatalities.fatality_rate.mode()

print("Fatality median is {} and mode is {}".format(fatality_median, fatality_mode))


Fatality median is 0.02194744976816074 and mode is 0    0.008403
dtype: float64


In [21]:
# the mode above does not make sense, let's fix it.

# first, segment and sort fatality values into bins
bins = pd.cut(fatalities['fatality_rate'], 10, include_lowest=True)

bins

# I'd like to see the content of each bin, what to do?

Country_Region
Italy                            (0.106, 0.117]
San Marino                       (0.106, 0.117]
Bangladesh                      (0.0943, 0.106]
Indonesia                      (0.0827, 0.0943]
Spain                          (0.0827, 0.0943]
                             ...               
Singapore      (0.00043000000000000004, 0.0131]
Uruguay        (0.00043000000000000004, 0.0131]
Qatar          (0.00043000000000000004, 0.0131]
Iceland        (0.00043000000000000004, 0.0131]
New Zealand    (0.00043000000000000004, 0.0131]
Name: fatality_rate, Length: 111, dtype: category
Categories (10, interval[float64]): [(0.00043000000000000004, 0.0131] < (0.0131, 0.0247] < (0.0247, 0.0363] < (0.0363, 0.0479] ... (0.0711, 0.0827] < (0.0827, 0.0943] < (0.0943, 0.106] < (0.106, 0.117]]

## Joined with Population data from United Nations 

Poplation is in thousand persons.  

In [22]:
population = pd.read_csv('data/WPP2019_TotalPopulationBySex.csv')
population.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
0,4,Afghanistan,2,Medium,1950,1950.5,4099.243,3652.874,7752.117,11.874
1,4,Afghanistan,2,Medium,1951,1951.5,4134.756,3705.395,7840.151,12.009
2,4,Afghanistan,2,Medium,1952,1952.5,4174.450,3761.546,7935.996,12.156
3,4,Afghanistan,2,Medium,1953,1953.5,4218.336,3821.348,8039.684,12.315
4,4,Afghanistan,2,Medium,1954,1954.5,4266.484,3884.832,8151.316,12.486


In [23]:
# we only need the data of year 2020
pop2019 = population[population.Time == 2019]
pop2019.head()

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
69,4,Afghanistan,2,Medium,2019,2019.5,19529.727,18512.030,38041.757,58.269
953,903,Africa,2,Medium,2019,2019.5,653513.680,654550.496,1308064.176,44.119
1837,1823,African Group,2,Medium,2019,2019.5,652644.714,653675.858,1306320.572,44.464
1988,1560,African Union,2,Medium,2019,2019.5,652949.469,653953.561,1306903.030,44.085
2139,2080,African Union: Central Africa,2,Medium,2019,2019.5,76945.498,77068.207,154013.705,29.192


In [24]:
len(pop2019)

477

### Now we need to unify the lation names of the population data and the corona data 

In [25]:
# The set of location names of the population data is much bigger than corona's. 
# So we first find out which country names are not in the population data

countryNames_covid19_only = df_country_cases[df_country_cases.index.isin(pop2019.Location) == False]

countryNames_covid19_only.index

Index(['Bolivia', 'Brunei', 'Burma', 'Congo (Brazzaville)', 'Congo (Kinshasa)',
       'Cote d'Ivoire', 'Diamond Princess', 'Iran', 'Korea, South', 'Kosovo',
       'Laos', 'MS Zaandam', 'Moldova', 'Russia', 'Syria', 'Taiwan*',
       'Tanzania', 'US', 'Venezuela', 'Vietnam', 'West Bank and Gaza'],
      dtype='object', name='Country_Region')

In [26]:
len(countryNames_covid19_only)

21

In [27]:
pop2019[pop2019.Location.str.contains('Pales')]

,LocID,Location,VarID,Variant,Time,MidPeriod,PopMale,PopFemale,PopTotal,PopDensity
236754,275,State of Palestine,2,Medium,2019,2019.5,2526.35,2455.072,4981.422,827.479


In [28]:
pop2019.Location.replace({'Bolivia (Plurinational State of)':'Bolivia', 'Brunei Darussalam':'Brunei', 'Myanmar': 'Burma', 'Congo':'Congo (Brazzaville)', 'Democratic Republic of the Congo':'Congo (Kinshasa)', 'Côte d\'Ivoire':'Cote d\'Ivoire', 'Iran (Islamic Republic of)':'Iran', 'Republic of Korea':'Korea, South', 'Lao People\'s Democratic Republic':'Laos', 'Republic of Moldova':'Moldova', 'Russian Federation':'Russia', 'Syrian Arab Republic':'Syria', 'China, Taiwan Province of China':'Taiwan*', 'United Republic of Tanzania':'Tanzania', 'United States of America':'US', 'Venezuela (Bolivarian Republic of)':'Venezuela', 'Viet Nam':'Vietnam', 'State of Palestine':'West Bank and Gaza'}, inplace=True)

In [29]:
# drop unnecessary columns

df_pop2019 = pop2019.drop(columns=['LocID','VarID','Variant','Time','MidPeriod'])
df_pop2019.head()

,Location,PopMale,PopFemale,PopTotal,PopDensity
69,Afghanistan,19529.727,18512.030,38041.757,58.269
953,Africa,653513.680,654550.496,1308064.176,44.119
1837,African Group,652644.714,653675.858,1306320.572,44.464
1988,African Union,652949.469,653953.561,1306903.030,44.085
2139,African Union: Central Africa,76945.498,77068.207,154013.705,29.192


In [30]:
# now join the corona and population data frames

df_covid19_pop2019 = pd.merge(df_country_cases, df_pop2019, how='inner', left_on='Country_Region', right_on='Location')
df_covid19_pop2019.head()

,Confirmed,Deaths,Recovered,Active,fatality_rate,Location,PopMale,PopFemale,PopTotal,PopDensity
0,174,4,5,165,0.022989,Afghanistan,19529.727,18512.030,38041.757,58.269
1,243,15,52,176,0.061728,Albania,1466.785,1414.128,2880.913,105.143
2,716,44,46,626,0.061453,Algeria,21749.666,21303.388,43053.054,18.076
3,376,12,10,354,0.031915,Andorra,NaN,NaN,77.146,164.140
4,7,2,1,4,0.285714,Angola,15744.779,16080.520,31825.299,25.528


In [31]:
df_covid19_pop2019.shape

(177, 10)

In [32]:
# correlation between fatality_rate and PopDensity??

corr_fatality_popDensity = df_covid19_pop2019.fatality_rate.corr(df_covid19_pop2019.PopDensity)
corr_fatality_popDensity

-0.039919797181260236